In [1]:
import pandas as pd
import numpy as np

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

In [11]:
data = pd.read_parquet("data\\arr_data.parquet")
data

,yr,mon,day_of_week,dep_hour,arr_hour,u_carrier,origin_airport_code,dest_airport_code,arr_delay_new,arr_del15,distance_grp
0,2003,1,3,9,11,US,PIT,LAX,0,0,9
1,2003,1,3,9,12,AA,JFK,LAX,0,0,10
2,2003,1,3,8,10,B6,BUF,JFK,0,0,2
3,2003,1,3,9,12,AS,DCA,SEA,2,0,10
4,2003,1,3,8,12,HP,PHX,ORD,0,0,6
...,...,...,...,...,...,...,...,...,...,...,...
68272049,2012,12,1,11,12,OO,MSP,IMT,0,0,2
68272050,2012,12,1,19,20,OO,MSP,IMT,0,0,2
68272051,2012,12,1,11,12,OO,MSP,INL,0,0,2
68272052,2012,12,1,11,12,OO,SLC,PIH,7,0,1


In [14]:
data.day_of_week.value_counts()

5    10096282
1    10074991
4    10073170
3     9948810
2     9856560
7     9611762
6     8610479
Name: day_of_week, dtype: int64

In [16]:
data.distance_grp.value_counts()

2     15088451
3     14534261
4     10413777
1      9176522
5      6172877
7      3603426
6      3269883
8      1842577
10     1632444
9      1376625
11     1161211
Name: distance_grp, dtype: int64

In [17]:
data.u_carrier.value_counts()

DL    7793108
AA    7636386
WN    6886425
UA    6451700
OO    5470910
US    4738552
MQ    4672792
NW    3654442
CO    3597984
EV    3110655
XE    3008087
FL    2636208
B6    1791810
AS    1427879
YV    1314839
OH     959887
F9     952402
HP     863347
9E     719028
DH     278119
TZ     153681
VX      93316
HA      59346
KH       1151
Name: u_carrier, dtype: int64

In [29]:
data.origin_airport_code.value_counts()

ATL    5055909
ORD    4452054
DFW    3765836
LAX    3157224
DEN    2981342
        ...   
SHD         18
PIR          8
OGD          5
CDV          3
PVU          1
Name: origin_airport_code, Length: 331, dtype: int64

In [30]:
data.dest_airport_code.value_counts()

ATL    5035476
ORD    4448764
DFW    3754987
LAX    3156084
DEN    2977174
        ...   
AZA         47
IMT         30
SHD         18
PIR          8
CBM          1
Name: dest_airport_code, Length: 329, dtype: int64

# origin and dest

In [36]:
# get pairwise count of each origin and dest
pairwise_count = data.groupby(['origin_airport_code', 'dest_airport_code']).count()
pairwise_count

yr  mon  day_of_week  dep_hour  \
origin_airport_code dest_airport_code                                   
ABE                 ABE                 0    0            0         0   
                    ABI                 0    0            0         0   
                    ABQ                 0    0            0         0   
                    ABR                 0    0            0         0   
                    ABY                 0    0            0         0   
...                                    ..  ...          ...       ...   
YUM                 VPS                 0    0            0         0   
                    WYS                 0    0            0         0   
                    XNA                 0    0            0         0   
                    YKM                 0    0            0         0   
                    YUM                 0    0            0         0   

                                       arr_hour  u_carrier  arr_delay_new  \
origin_airport_code dest_airport_code                                       
ABE                 ABE                       0          0              0   
                    ABI                       0          0              0   
                    ABQ                       0          0              0   
                    ABR                       0          0              0   
                    ABY                       0          0              0   
...                                         ...        ...            ...   
YUM                 VPS                       0          0              0   
                    WYS                       0          0              0   
                    XNA                       0          0              0   
                    YKM                       0          0              0   
                    YUM                       0          0              0   

                                       arr_del15  distance_grp  
origin_airport_code dest_airport_code                           
ABE                 ABE                        0             0  
                    ABI                        0             0  
                    ABQ                        0             0  
                    ABR                        0             0  
                    ABY                        0             0  
...                                          ...           ...  
YUM                 VPS                        0             0  
                    WYS                        0             0  
                    XNA                        0             0  
                    YKM                        0             0  
                    YUM                        0             0  

[108899 rows x 9 columns]

In [38]:
# removing zero rows
pairwise_count = pairwise_count[pairwise_count.yr > 0]

In [46]:
airport_pairs = pairwise_count.reset_index()[["origin_airport_code", "dest_airport_code"]]
airport_pairs

,origin_airport_code,dest_airport_code
0,ABE,ATL
1,ABE,BWI
2,ABE,CLT
3,ABE,DTW
4,ABE,JFK
...,...,...
4277,YKM,SLC
4278,YUM,LAS
4279,YUM,LAX
4280,YUM,PHX


In [60]:
sorted(airport_pairs.dest_airport_code.unique().tolist())

['ABE',
 'ABI',
 'ABQ',
 'ABR',
 'ABY',
 'ACK',
 'ACT',
 'ACV',
 'ACY',
 'AEX',
 'AGS',
 'ALB',
 'ALO',
 'AMA',
 'ANC',
 'APF',
 'APN',
 'ART',
 'ASE',
 'ATL',
 'ATW',
 'AUS',
 'AVL',
 'AVP',
 'AZA',
 'AZO',
 'BDL',
 'BFL',
 'BGM',
 'BGR',
 'BHM',
 'BIL',
 'BIS',
 'BJI',
 'BKG',
 'BLI',
 'BMI',
 'BNA',
 'BOI',
 'BOS',
 'BPT',
 'BQK',
 'BQN',
 'BRD',
 'BRO',
 'BTM',
 'BTR',
 'BTV',
 'BUF',
 'BUR',
 'BWI',
 'BZN',
 'CAE',
 'CAK',
 'CBM',
 'CDC',
 'CEC',
 'CHA',
 'CHO',
 'CHS',
 'CIC',
 'CID',
 'CLD',
 'CLE',
 'CLL',
 'CLT',
 'CMH',
 'CMI',
 'CMX',
 'COD',
 'COS',
 'COU',
 'CPR',
 'CRP',
 'CRW',
 'CSG',
 'CVG',
 'CWA',
 'CYS',
 'DAB',
 'DAL',
 'DAY',
 'DBQ',
 'DCA',
 'DEN',
 'DFW',
 'DHN',
 'DLH',
 'DRO',
 'DRT',
 'DSM',
 'DTW',
 'EAU',
 'ECP',
 'EFD',
 'EGE',
 'EKO',
 'ELM',
 'ELP',
 'ERI',
 'ESC',
 'EUG',
 'EVV',
 'EWN',
 'EWR',
 'EYW',
 'FAI',
 'FAR',
 'FAT',
 'FAY',
 'FCA',
 'FLG',
 'FLL',
 'FLO',
 'FNT',
 'FSD',
 'FSM',
 'FWA',
 'GCC',
 'GCK',
 'GEG',
 'GFK',
 'GGG',
 'GJT',
 'GNV',


In [55]:
airport_pairs[airport_pairs.origin_airport_code == "ATL"][
            "dest_airport_code"
        ].tolist()

['ABE',
 'ABQ',
 'ABY',
 'ACY',
 'AEX',
 'AGS',
 'ALB',
 'ANC',
 'APF',
 'ASE',
 'ATW',
 'AUS',
 'AVL',
 'AVP',
 'AZO',
 'BDL',
 'BGM',
 'BGR',
 'BHM',
 'BIL',
 'BKG',
 'BMI',
 'BNA',
 'BOI',
 'BOS',
 'BPT',
 'BQK',
 'BQN',
 'BTR',
 'BTV',
 'BUF',
 'BUR',
 'BWI',
 'BZN',
 'CAE',
 'CAK',
 'CBM',
 'CHA',
 'CHO',
 'CHS',
 'CID',
 'CLE',
 'CLT',
 'CMH',
 'CMI',
 'COS',
 'COU',
 'CRP',
 'CRW',
 'CSG',
 'CVG',
 'DAB',
 'DAL',
 'DAY',
 'DCA',
 'DEN',
 'DFW',
 'DHN',
 'DSM',
 'DTW',
 'ECP',
 'EGE',
 'ELP',
 'ERI',
 'EVV',
 'EWN',
 'EWR',
 'EYW',
 'FAY',
 'FCA',
 'FLL',
 'FLO',
 'FNT',
 'FSD',
 'FSM',
 'FWA',
 'GNV',
 'GPT',
 'GRB',
 'GRK',
 'GRR',
 'GSO',
 'GSP',
 'GTR',
 'GUC',
 'HDN',
 'HHH',
 'HKY',
 'HNL',
 'HOU',
 'HPN',
 'HSV',
 'HTS',
 'IAD',
 'IAH',
 'ICT',
 'ILG',
 'ILM',
 'IND',
 'ISO',
 'ISP',
 'JAC',
 'JAN',
 'JAX',
 'JFK',
 'LAN',
 'LAS',
 'LAW',
 'LAX',
 'LEX',
 'LFT',
 'LGA',
 'LGB',
 'LIT',
 'LNK',
 'LWB',
 'LYH',
 'MCI',
 'MCN',
 'MCO',
 'MDT',
 'MDW',
 'MEI',
 'MEM',
 'MFE',


# distance grp

In [63]:
# sanity check
pairwise_dist_groups = data.groupby(['origin_airport_code', 'dest_airport_code']).distance_grp.nunique()
pairwise_dist_groups = pairwise_dist_groups[pairwise_dist_groups > 0]
pairwise_dist_groups.value_counts()

origin_airport_code  dest_airport_code
ABE                  ATL                  1
                     BWI                  1
                     CLT                  1
                     DTW                  1
                     JFK                  1
                                         ..
YKM                  SLC                  1
YUM                  LAS                  1
                     LAX                  1
                     PHX                  1
                     SLC                  1
Name: distance_grp, Length: 4282, dtype: int64

In [70]:
airport_pairs_dist = data[[
    'origin_airport_code', 'dest_airport_code', 'distance_grp'
]].drop_duplicates()

In [72]:
airport_pairs_dist.to_csv("data/airport_pairs.csv", index = False)

In [75]:
airport_pairs = pd.read_csv("data/airport_pairs.csv")
airport_pairs

,origin_airport_code,dest_airport_code,distance_grp
0,PIT,LAX,9
1,JFK,LAX,10
2,BUF,JFK,2
3,DCA,SEA,10
4,PHX,ORD,6
...,...,...,...
4277,MCO,SHD,3
4278,MSP,IMT,2
4279,JFK,PSP,10
4280,LGA,EYW,5


In [79]:
airport_pairs[(airport_pairs.origin_airport_code == "YUM") &
    (airport_pairs.dest_airport_code == "SLC")]['distance_grp'].values[0]

3

# carriers

In [84]:
# get carriers
print(sorted(data.u_carrier.unique()))

['9E', 'AA', 'AS', 'B6', 'CO', 'DH', 'DL', 'EV', 'F9', 'FL', 'HA', 'HP', 'KH', 'MQ', 'NW', 'OH', 'OO', 'TZ', 'UA', 'US', 'VX', 'WN', 'XE', 'YV']


# making options dictionary

In [85]:
# writing options to dictionary
import json
options_dict = {"dayofweek": [{'label':'Mon', 'value':1}, {'label':'Tue','value':2}, {'label':'Wed','value':3},
                 {'label':'Thu','value':4}, {'label':'Fri','value':5}, {'label':'Sat','value':6}, {'label':'Sun','value':7}],
                "year": [2015, 2014, 2013], #2022, 2021, 2020, 2019, 2018, 2017, 2016, 
               "month": [
                {'label':'Jan', 'value':1},
                {'label':'Feb','value':2},
                {'label':'Mar','value':3},
                {'label':'Apr','value':4},
                {'label':'May','value':5},
                {'label':'Jun','value':6},
                {'label':'Jul','value':7},
                {'label':'Aug','value':8},
                {'label':'Sep','value':9},
                {'label':'Oct','value':10},
                {'label':'Nov','value':11},
                {'label':'Dec','value':12},                
            ],
               "carrier": ['9E', 'AA', 'AS', 'B6', 'CO', 'DH', 'DL', 'EV', 'F9', 'FL', 'HA', 
                           'HP', 'KH', 'MQ', 'NW', 'OH', 'OO', 'TZ', 'UA', 'US', 'VX', 'WN', 'XE', 'YV']}

with open("data/options_dict.txt","w") as file:
  file.write(json.dumps(options_dict))

In [86]:
# check dictionary content
with open("data/options_dict.txt", "r") as file:
 file_content = file.read()
eval(file_content)

{'dayofweek': [{'label': 'Mon', 'value': 1},
  {'label': 'Tue', 'value': 2},
  {'label': 'Wed', 'value': 3},
  {'label': 'Thu', 'value': 4},
  {'label': 'Fri', 'value': 5},
  {'label': 'Sat', 'value': 6},
  {'label': 'Sun', 'value': 7}],
 'year': [2015, 2014, 2013],
 'month': [{'label': 'Jan', 'value': 1},
  {'label': 'Feb', 'value': 2},
  {'label': 'Mar', 'value': 3},
  {'label': 'Apr', 'value': 4},
  {'label': 'May', 'value': 5},
  {'label': 'Jun', 'value': 6},
  {'label': 'Jul', 'value': 7},
  {'label': 'Aug', 'value': 8},
  {'label': 'Sep', 'value': 9},
  {'label': 'Oct', 'value': 10},
  {'label': 'Nov', 'value': 11},
  {'label': 'Dec', 'value': 12}],
 'carrier': ['9E',
  'AA',
  'AS',
  'B6',
  'CO',
  'DH',
  'DL',
  'EV',
  'F9',
  'FL',
  'HA',
  'HP',
  'KH',
  'MQ',
  'NW',
  'OH',
  'OO',
  'TZ',
  'UA',
  'US',
  'VX',
  'WN',
  'XE',
  'YV']}